<a href="https://colab.research.google.com/github/dimaboyko/tensorflow_examples/blob/master/pandas_for_beginners_02_joins_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In real life, we often have one-to-many relations between things:

* A partner can have many customers.
* A customer can have many accounts.
* An account can have many transaction.

How is such information stored in databases? You could store it "denormalized" like this:

Transactions:
* Roland Folz, age 50, transfers 50€ from his account DE123456 to DE9876
* Roland Folz, age 50, transfers 123€ from his account DE123456 to DE444
* Roland Folz, age 50, transfers 234€ from his account DE123456 to DE555
* Roland Folz, age 50, transfers 345€ from his account DE123456 to DE666

Bad idea because:
* You store a lot of duplicate data!
* To just get a list of all customers, you need to read through ALL transactions ever made.

Instead, a process called "normalization" during database design transforms it into this:

Customers:
* Customer 1 is Roland Folz, age 50

Accounts:
* Account 5 belongs to customer 1, IBAN is DE123456

Transactions:
* 50€ from account 5 to DE9876
* 123€ from account 5 to DE444
* 234€ from account 5 to DE555
* 345€ from account 5 to DE666

Very nice, but...

...if you want know "all transactions of Roland Folz", you first need to put the pieces back together!

In [1]:
from pandas import DataFrame

customers = [
    {'customer_id': 1, 'first_name': 'Roland', 'last_name': 'Folz', 'age': 50, 'is_pep': False},
    {'customer_id': 4, 'first_name': 'Roland', 'last_name': 'Emmerich', 'age': 64, 'is_pep': False},
    {'customer_id': 12, 'first_name': 'Ronald', 'last_name': 'McDonald', 'age': 99, 'is_pep': False},
    {'customer_id': 15, 'first_name': 'Ronald', 'last_name': 'Reagan', 'age': 108, 'is_pep': True},
    {'customer_id': 16, 'first_name': 'Ronaldo', 'last_name': 'Nazário de Lima', 'age': 42, 'is_pep': False},
]

customers = DataFrame(customers)
customers

,age,customer_id,first_name,is_pep,last_name
0,50,1,Roland,False,Folz
1,64,4,Roland,False,Emmerich
2,99,12,Ronald,False,McDonald
3,108,15,Ronald,True,Reagan
4,42,16,Ronaldo,False,Nazário de Lima


In [2]:
accounts = [
    {'account_id': 1, 'customer_id': 1, 'balance': 4_242},
    {'account_id': 5, 'customer_id': 1, 'balance': 424_242},
    {'account_id': 14, 'customer_id': 4, 'balance': 1_000_000},
    {'account_id': 17, 'customer_id': 15, 'balance': 100_000_000},
    {'account_id': 23, 'customer_id': 16, 'balance': 10_000},
    {'account_id': 42, 'customer_id': 16, 'balance': 10_000_000},
    {'account_id': 43, 'customer_id': 16, 'balance': 42_000_000},
]

accounts = DataFrame(accounts)
accounts

,account_id,balance,customer_id
0,1,4242,1
1,5,424242,1
2,14,1000000,4
3,17,100000000,15
4,23,10000,16
5,42,10000000,16
6,43,42000000,16


In [3]:
accounts_and_customers = customers.merge(accounts)
accounts_and_customers

,age,customer_id,first_name,is_pep,last_name,account_id,balance
0,50,1,Roland,False,Folz,1,4242
1,50,1,Roland,False,Folz,5,424242
2,64,4,Roland,False,Emmerich,14,1000000
3,108,15,Ronald,True,Reagan,17,100000000
4,42,16,Ronaldo,False,Nazário de Lima,23,10000
5,42,16,Ronaldo,False,Nazário de Lima,42,10000000
6,42,16,Ronaldo,False,Nazário de Lima,43,42000000


How did `merge()` know how to combine customers and account?

Both DataFrames have a `customer_id` column.

If columns have different names, use `left_on=` and `right_on=` to specify the columns


Note that Ronald McDonald does not appear in this table, because he has no account!

In [4]:
transactions = [
    {'transaction_id': 100, 'account_id': 17, 'amount': 6_000, 'to_iban': 'DE123'},
    {'transaction_id': 101, 'account_id': 1, 'amount': 4_000, 'to_iban': 'DE234'},
    {'transaction_id': 102, 'account_id': 42, 'amount': 1_000_000, 'to_iban': 'BR345'},
    {'transaction_id': 103, 'account_id': 14, 'amount': 42_000, 'to_iban': 'DE456'},
    {'transaction_id': 104, 'account_id': 5, 'amount': 123, 'to_iban': 'AT123'},
    {'transaction_id': 105, 'account_id': 5, 'amount': 234, 'to_iban': 'DE567'},
    {'transaction_id': 106, 'account_id': 23, 'amount': 999, 'to_iban': 'FR456'},
]

transactions = DataFrame(transactions)
transactions

,account_id,amount,to_iban,transaction_id
0,17,6000,DE123,100
1,1,4000,DE234,101
2,42,1000000,BR345,102
3,14,42000,DE456,103
4,5,123,AT123,104
5,5,234,DE567,105
6,23,999,FR456,106


In [5]:
transactions_merged = accounts_and_customers.merge(transactions)
transactions_merged

,age,customer_id,first_name,is_pep,last_name,account_id,balance,amount,to_iban,transaction_id
0,50,1,Roland,False,Folz,1,4242,4000,DE234,101
1,50,1,Roland,False,Folz,5,424242,123,AT123,104
2,50,1,Roland,False,Folz,5,424242,234,DE567,105
3,64,4,Roland,False,Emmerich,14,1000000,42000,DE456,103
4,108,15,Ronald,True,Reagan,17,100000000,6000,DE123,100
5,42,16,Ronaldo,False,Nazário de Lima,23,10000,999,FR456,106
6,42,16,Ronaldo,False,Nazário de Lima,42,10000000,1000000,BR345,102


This time, the `merge()` worked because both DataFrames have a `account_id` column.

# Exercise
The AML department needs to perform special AML checks if one of the following is true:

*  transaction over 10,000 €
*  for PEPs: transaction over 5,000€
*  transaction that move >90% of the account's (original) balance
*  transaction going outside of Germany

We need to support them and get all the transaction IDs for all matching transactions.

Also, we need to tell the AML team the reason(s) why a transaction needs to be checked.

That is a lot of stuff to do. Action plan:
* implement all four conditions
 * one by one!
 * add a new column to the DataFrame for each condition
* use the new columns to generate a description of what needs to be checked
* those items that have a description must be given to the AML team


First condition: transactions over 10,000€

Add a new column `is_high_transaction` to `transactions_merged` that tells us if the transaction amount is greater than 10,000€.

In [0]:
transactions_merged['is_high_transaction'] = transactions_merged.amount > 10000

In [7]:
transactions_merged

,age,customer_id,first_name,is_pep,last_name,account_id,balance,amount,to_iban,transaction_id,is_high_transaction
0,50,1,Roland,False,Folz,1,4242,4000,DE234,101,False
1,50,1,Roland,False,Folz,5,424242,123,AT123,104,False
2,50,1,Roland,False,Folz,5,424242,234,DE567,105,False
3,64,4,Roland,False,Emmerich,14,1000000,42000,DE456,103,True
4,108,15,Ronald,True,Reagan,17,100000000,6000,DE123,100,False
5,42,16,Ronaldo,False,Nazário de Lima,23,10000,999,FR456,106,False
6,42,16,Ronaldo,False,Nazário de Lima,42,10000000,1000000,BR345,102,True


Second condition: transactions over 5,000€ if customer is a PEP

Add a new column `is_high_transaction_pep` to `transactions_merged` that tells us if the condition applies.

Hint: If it says `ValueError: The truth value of a Series is ambiguous` then use parentheses to clarify things.

In [8]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Jul 19 16:14 .
drwxr-xr-x 1 root root 4096 Jul 25 13:21 ..
drwxr-xr-x 1 root root 4096 Jul 19 16:14 .config
drwxr-xr-x 1 root root 4096 Jul 19 16:14 sample_data


In [0]:
# TODO: add new column to transactions_merged

Third condition: transaction to account outside of Germany

German IBANs start with `DE`, so we look for `to_iban`s that do *not* start with `DE`.

In [0]:
# normal Python:
iban = "FR1234"
iban.startswith("DE")

In [0]:
# Pandas:
transactions_merged.first_name.str.startswith("Rol")

Insert a new column `to_germany` that indicates if the transaction goes to Germany or not.

In [0]:
# TODO: add new column to transactions_merged

Fourth condition: transfer more than 90% of balance

Insert a new column `is_high_percentage` that indicates if the condition matches.

Note: Remember that mathematical operations on a column are automatically applied to all numbers in the column. You do *not* need a for loop.

In [0]:
transactions_merged['is_high_percentage'] = (transactions_merged["amount"] / transactions_merged["balance"]) > 0.9

transactions_merged

Now we need to put all the pieces together and tell the AML team why the transaction needs to be checked. There can be one or multiple reasons why a check is needed!

Use the `apply()` method you saw last week to create a new column `aml_check_reason`. Combine `apply()` with the function `get_aml_check_reason()` (which you also need to implement).

In [0]:
def get_aml_check_reason(transaction):
    # TODO: return a string with the reasons why
    #       the transaction needs to be checked.

In [0]:
# TODO: add new column to transactions_merged by using apply()

In [0]:
# Finally, let's get the transactions that need checking:
to_be_checked = transactions_merged[transactions_merged.aml_check_reason != ""]

# We only want two columns:
to_be_checked[['transaction_id', 'aml_check_reason']]